У нас есть фичи, в которых очень много пропусков.

И есть лейблы, в которых очень много нулей.

Может быть, тут есть связь?

In [8]:
import numpy as np
import pandas as pd
import sklearn

from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [33]:
from IPython.core.display import HTML

HTML(
    r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: "JetBrainsMono Nerd Font Mono"; # Any monospaced font should work
    }
</style>
"""
)

np.set_printoptions(precision=2)
pd.set_option("display.precision", 2)
pd.set_option("display.float_format", "{:.2f}".format)
seed = 42

In [5]:
train_df = pd.read_csv("./data/prepared/train.csv", low_memory=False).drop(
    columns=["id"]
)
test_df = pd.read_csv("./data/prepared/test.csv", low_memory=False).drop(columns=["id"])
train_labels_df = pd.read_csv(
    "./data/prepared/train_labels.csv", low_memory=False
).drop(columns=["id"])

In [4]:
train_counts = train_df.describe(include="all").loc["count"]
train_counts

release      8000
n_0000       12.0
n_0001      388.0
n_0002     7662.0
n_0003      112.0
            ...  
c_1373        440
c_1374       3155
c_1375        563
c_1376          7
c_1377       1908
Name: count, Length: 1194, dtype: object

И как я это буду проверятть?

In [7]:
train_labels_df.apply(pd.value_counts)

,service_a,service_b,service_c,service_d,service_e,service_f,service_g,service_h,service_i,service_j,service_k,service_l,service_m,service_n
0,4223,5374,5907,7864,7569,7767,7585,5624,7871,1214,1744,7136,7291,6576
1,3777,2626,2093,136,431,233,415,2376,129,6786,6256,864,709,1424


Возьмем в качестве жертвы `service_i`

In [16]:
temp = train_df.copy()
temp["service_i"] = train_labels_df["service_i"]
label_corr = temp.corr(numeric_only=True)["service_i"].drop(labels=["service_i"])
label_corr[label_corr > 0.1]

n_0007    0.15
n_0023    0.47
n_0036    0.30
n_0095    0.39
o_0130    0.13
o_0148    0.14
o_0160    0.18
o_0207    0.14
o_0222    0.14
o_0295    0.11
Name: service_i, dtype: float64

 у n_0023 достаточно высокая корреляция 🤔

In [17]:
train_df["n_0023"].value_counts()

0.0    18
0.5     4
1.0     3
Name: n_0023, dtype: int64

При этом он очень плохо представлен — 25 из 8000

In [21]:
train_df[label_corr[label_corr > 0.1].index].count()

n_0007     995
n_0023      25
n_0036      55
n_0095    1684
o_0130      38
o_0148     109
o_0160      39
o_0207      32
o_0222     146
o_0295     102
dtype: int64

## попробуем добавить в датасет feature_is_na

In [7]:
data = pd.get_dummies(train_df)
data = SimpleImputer(add_indicator=True).fit_transform(data)

In [15]:
pd.get_dummies(train_df).shape

(8000, 5605)

In [14]:
data.shape

(8000, 5876)

In [16]:
clf.fit(data, train_labels_df)

MultiOutputClassifier(estimator=LogisticRegression(random_state=42,
                                                   solver='liblinear'))

In [20]:
train_features, valid_features, train_targets, valid_targets = train_test_split(
    data, train_labels_df, random_state=seed, test_size=0.2
)

In [23]:
clf = MultiOutputClassifier(LogisticRegression(solver="liblinear", random_state=seed))
clf.fit(train_features, train_targets)
valid_probas = clf.predict_proba(valid_features)

{
    label: round(
        metrics.log_loss(valid_targets.iloc[:, label_id], valid_probas[label_id]),
        2,
    )
    for label_id, label in enumerate(train_labels_df.columns)
}

{'service_a': 0.55,
 'service_b': 0.68,
 'service_c': 0.74,
 'service_d': 0.05,
 'service_e': 0.29,
 'service_f': 0.12,
 'service_g': 0.35,
 'service_h': 0.71,
 'service_i': 0.1,
 'service_j': 0.59,
 'service_k': 0.71,
 'service_l': 0.13,
 'service_m': 0.15,
 'service_n': 0.59}

работает плохо

## посмотрим на корреляцию между is_na и лейблами

In [26]:
feature_is_na = train_df.isna()
feature_is_na

,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,n_0009,...,c_1366,c_1367,c_1369,c_1370,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
0,False,True,True,False,True,True,False,True,True,True,...,True,True,True,True,False,True,False,True,True,True
1,False,True,True,False,True,True,False,True,True,True,...,True,True,True,False,False,True,True,True,True,True
2,False,True,True,False,True,True,False,True,True,True,...,True,True,True,False,False,True,False,True,True,True
3,False,True,True,False,True,True,False,True,True,True,...,True,True,True,True,False,True,True,True,True,True
4,False,True,True,False,True,True,False,True,True,True,...,True,True,True,False,False,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,False,True,True,False,True,True,False,True,True,True,...,False,True,True,True,False,True,True,True,True,False
7996,False,True,True,False,True,True,False,True,True,True,...,True,True,True,True,False,True,False,True,True,True
7997,False,True,False,False,True,True,False,True,True,True,...,True,True,True,True,False,True,True,True,True,True
7998,False,True,True,False,True,True,False,True,True,True,...,True,True,True,True,False,True,False,True,True,True


In [37]:
na_with_labels = pd.concat((feature_is_na, train_labels_df), axis=1)
corr = na_with_labels.corr()[train_labels_df.columns].abs()

In [39]:
corr

,service_a,service_b,service_c,service_d,service_e,service_f,service_g,service_h,service_i,service_j,service_k,service_l,service_m,service_n
release,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
n_0000,0.01,0.00,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.02
n_0001,0.08,0.05,0.05,0.03,0.00,0.00,0.02,0.02,0.02,0.04,0.05,0.02,0.03,0.01
n_0002,0.00,0.02,0.07,0.02,0.03,0.04,0.01,0.12,0.03,0.24,0.17,0.07,0.07,0.07
n_0003,0.02,0.04,0.01,0.00,0.01,0.01,0.00,0.01,0.02,0.01,0.02,0.01,0.01,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
service_j,0.06,0.06,0.05,0.00,0.03,0.06,0.03,0.01,0.00,1.00,0.53,0.01,0.00,0.03
service_k,0.07,0.04,0.02,0.02,0.04,0.06,0.05,0.05,0.00,0.53,1.00,0.08,0.07,0.04
service_l,0.04,0.03,0.13,0.15,0.17,0.01,0.01,0.31,0.04,0.01,0.08,1.00,0.80,0.12
service_m,0.06,0.05,0.14,0.17,0.17,0.02,0.01,0.21,0.02,0.00,0.07,0.80,1.00,0.09


relase NaN потому что он определен на всех записях

In [40]:
label = "service_a"
corr[label][corr[label] > 0.5]

c_0572      0.57
service_a   1.00
service_b   0.56
Name: service_a, dtype: float64

In [43]:
train_df["c_0572"].value_counts()

b    1543
g     128
d     111
c      75
f      38
a      15
Name: c_0572, dtype: int64

проверим, насколько хорошо работает чисто эта фича

In [49]:
nan_predictor = LogisticRegression()
cross_val_score(nan_predictor, feature_is_na[["c_0572"]], train_labels_df["service_a"], scoring="roc_auc")

array([0.73, 0.75, 0.74, 0.75, 0.74])

In [50]:
cross_val_score(nan_predictor, feature_is_na[["c_0572"]], train_labels_df["service_a"], scoring="neg_log_loss")

array([-0.51, -0.49, -0.5 , -0.5 , -0.51])

lol

В бейзлайне у нас roc auc 0.91, log_loss 0.37. а у service_b максимально выбило 0.49

пройдемся по другим колонкам

In [53]:
label = "service_b"
corr[label][corr[label] > 0.3]

c_0572      0.36
service_a   0.56
service_b   1.00
service_c   0.41
Name: service_b, dtype: float64

Какая знакомая колонка 🤔

In [57]:
cross_val_score(LogisticRegression(), feature_is_na[["c_0572"]], train_labels_df["service_b"], scoring="roc_auc")

array([0.65, 0.67, 0.66, 0.67, 0.66])

In [58]:
cross_val_score(LogisticRegression(), feature_is_na[["c_0572"]], train_labels_df["service_b"], scoring="neg_log_loss")

array([-0.58, -0.57, -0.57, -0.57, -0.57])

интересно, сколько таких колонок нужно добавить чтобы получить скор лучше бейзлайна

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [94]:
train_counts = train_df.describe(include="all").loc["count"]
well_defined_columns = train_counts[train_counts > 7000].index
well_done_df = train_df[well_defined_columns]

well_done_df = well_done_df.reindex(columns=sorted(well_done_df.columns, key=lambda col: well_done_df.dtypes[col]))
cat_features = np.where(well_done_df.dtypes == object)[0]
numerical_features = np.where(well_done_df.dtypes != object)[0]

In [100]:
pipeline = Pipeline(
    (
        (
            "impute",
            ColumnTransformer(
                (
                    (
                        "impute_numerical",
                        SimpleImputer(strategy="mean"),
                        numerical_features,
                    ),
                    (
                        "impute_categorical",
                        SimpleImputer(strategy="most_frequent"),
                        cat_features,
                    ),
                ),
                remainder="passthrough",
            ),
        ),
        (
            "one_hot_categorical",
            ColumnTransformer(
                (("inner", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_features),),
                remainder="passthrough",
            ),
        ),
        ("model", LogisticRegression(solver="liblinear", random_state=seed)),
    )
)

In [73]:
cross_val_score(
    pipeline,
    well_done_df,
    train_labels_df["service_b"],
    scoring="neg_log_loss",
)

array([-0.53, -0.52, -0.52, -0.53, -0.51])

In [74]:
cross_val_score(
    pipeline,
    pd.concat((well_done_df, feature_is_na[["c_0572"]]), axis=1),
    train_labels_df["service_b"],
    scoring="neg_log_loss",
)

array([-0.53, -0.52, -0.52, -0.53, -0.51])

не улучшается

In [97]:
cross_val_score(
    pipeline,
    well_done_df,
    train_labels_df["service_a"],
    scoring="neg_log_loss",
)

array([-0.42, -0.4 , -0.42, -0.42, -0.42])

In [98]:
cross_val_score(
    pipeline,
    pd.concat((well_done_df, feature_is_na[["c_0572"]]), axis=1),
    train_labels_df["service_a"],
    scoring="neg_log_loss",
)

array([-0.42, -0.4 , -0.42, -0.42, -0.42])

Попробуем засунуть все isna

In [106]:
feature_is_na.columns = [f"{name}_isna" for name in train_df.columns]

In [109]:
well_done_with_isna = pd.concat((well_done_df, feature_is_na), axis=1)
well_done_with_isna = well_done_with_isna.reindex(
    columns=sorted(
        well_done_with_isna.columns, key=lambda col: well_done_with_isna.dtypes[col]
    )
)
cat_features = np.where(well_done_with_isna.dtypes == object)[0]
numerical_features = np.where(well_done_with_isna.dtypes != object)[0]

pipeline = Pipeline(
    (
        (
            "impute",
            ColumnTransformer(
                (
                    (
                        "impute_numerical",
                        SimpleImputer(strategy="mean"),
                        numerical_features,
                    ),
                    (
                        "impute_categorical",
                        SimpleImputer(strategy="most_frequent"),
                        cat_features,
                    ),
                ),
                remainder="passthrough",
            ),
        ),
        (
            "one_hot_categorical",
            ColumnTransformer(
                (
                    (
                        "inner",
                        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
                        cat_features,
                    ),
                ),
                remainder="passthrough",
            ),
        ),
        # ("model", LogisticRegression(solver="liblinear", random_state=seed)),
        ("model", RandomForestClassifier(random_state=seed)),
    )
)

cross_val_score(
    pipeline,
    well_done_with_isna,
    train_labels_df["service_a"],
    scoring="neg_log_loss",
)

array([-0.45, -0.44, -0.44, -0.45, -0.45])

Наверное, слишком сложно

In [110]:
well_done_with_isna

,release_isna,n_0000_isna,n_0001_isna,n_0002_isna,n_0003_isna,n_0004_isna,n_0005_isna,n_0006_isna,n_0007_isna,n_0009_isna,...,c_1223,c_1227,c_1236,c_1244,c_1252,c_1259,c_1286,c_1316,c_1348,c_1372
0,False,True,True,False,True,True,False,True,True,True,...,c,a,c,d,b,n,b,b,b,a
1,False,True,True,False,True,True,False,True,True,True,...,a,a,c,d,b,e,b,b,b,a
2,False,True,True,False,True,True,False,True,True,True,...,c,a,a,d,b,w,b,b,b,a
3,False,True,True,False,True,True,False,True,True,True,...,c,a,c,d,b,e,b,a,b,a
4,False,True,True,False,True,True,False,True,True,True,...,c,a,c,d,b,e,b,b,b,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,False,True,True,False,True,True,False,True,True,True,...,c,b,c,a,b,i,a,a,c,a
7996,False,True,True,False,True,True,False,True,True,True,...,c,a,c,d,b,w,b,b,b,a
7997,False,True,False,False,True,True,False,True,True,True,...,c,a,c,d,b,n,b,b,b,a
7998,False,True,True,False,True,True,False,True,True,True,...,c,a,a,a,b,n,b,b,b,a
